In [6]:
import pandas as pd
# Read dataset from 2015
immigrants = pd.read_csv('migration_flows_bilateral_2015_clean.csv')

In [2]:
immigrants.head()

,YEAR,COUNTRY,EMIGRANTS_FROM_AFGHANISTAN,IMMIGRANTS_TO_AFGHANISTAN,EMIGRANTS_FROM_ALBANIA,IMMIGRANTS_TO_ALBANIA,EMIGRANTS_FROM_ALGERIA,IMMIGRANTS_TO_ALGERIA,EMIGRANTS_FROM_AMERICAN_SAMOA,IMMIGRANTS_TO_AMERICAN_SAMOA,...,EMIGRANTS_FROM_ZIMBABWE,IMMIGRANTS_TO_ZIMBABWE,COTEDIVOIRE_DESTINATION,MICRONESIACOUNTRY_DESTINATION,SAINTMARTINFRENCHPART_DESTINATION,SINTMAARTENDUTCHPART_DESTINATION,COTEDIVOIRE_ORIGIN,MICRONESIACOUNTRY_ORIGIN,SAINTMARTINFRENCHPART_ORIGIN,SINTMAARTENDUTCHPART_ORIGIN
0,2015,AFGHANISTAN,-5400916.0,339432.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
1,2015,ALBANIA,NaN,NaN,-1142844.0,52031.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
2,2015,ALGERIA,NaN,NaN,NaN,NaN,-1836502.0,239473.0,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
3,2015,AMERICAN SAMOA,NaN,NaN,NaN,NaN,NaN,NaN,-1961.0,23513.0,...,NaN,NaN,NaN,-23,NaN,NaN,NaN,0,NaN,NaN
4,2015,ANDORRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN


In [3]:
emigrants = immigrants.copy()
emigrants.head()

,YEAR,COUNTRY,EMIGRANTS_FROM_AFGHANISTAN,IMMIGRANTS_TO_AFGHANISTAN,EMIGRANTS_FROM_ALBANIA,IMMIGRANTS_TO_ALBANIA,EMIGRANTS_FROM_ALGERIA,IMMIGRANTS_TO_ALGERIA,EMIGRANTS_FROM_AMERICAN_SAMOA,IMMIGRANTS_TO_AMERICAN_SAMOA,...,EMIGRANTS_FROM_ZIMBABWE,IMMIGRANTS_TO_ZIMBABWE,COTEDIVOIRE_DESTINATION,MICRONESIACOUNTRY_DESTINATION,SAINTMARTINFRENCHPART_DESTINATION,SINTMAARTENDUTCHPART_DESTINATION,COTEDIVOIRE_ORIGIN,MICRONESIACOUNTRY_ORIGIN,SAINTMARTINFRENCHPART_ORIGIN,SINTMAARTENDUTCHPART_ORIGIN
0,2015,AFGHANISTAN,-5400916.0,339432.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
1,2015,ALBANIA,NaN,NaN,-1142844.0,52031.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
2,2015,ALGERIA,NaN,NaN,NaN,NaN,-1836502.0,239473.0,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
3,2015,AMERICAN SAMOA,NaN,NaN,NaN,NaN,NaN,NaN,-1961.0,23513.0,...,NaN,NaN,NaN,-23,NaN,NaN,NaN,0,NaN,NaN
4,2015,ANDORRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN


In [4]:
immigrants = immigrants[immigrants.columns.drop(list(immigrants.filter(regex='EMIGRANTS')))]
immigrants.head()

,YEAR,COUNTRY,IMMIGRANTS_TO_AFGHANISTAN,IMMIGRANTS_TO_ALBANIA,IMMIGRANTS_TO_ALGERIA,IMMIGRANTS_TO_AMERICAN_SAMOA,IMMIGRANTS_TO_ANDORRA,IMMIGRANTS_TO_ANGOLA,IMMIGRANTS_TO_ANGUILLA,IMMIGRANTS_TO_ANTIGUA_AND_BARBUDA,...,IMMIGRANTS_TO_ZAMBIA,IMMIGRANTS_TO_ZIMBABWE,COTEDIVOIRE_DESTINATION,MICRONESIACOUNTRY_DESTINATION,SAINTMARTINFRENCHPART_DESTINATION,SINTMAARTENDUTCHPART_DESTINATION,COTEDIVOIRE_ORIGIN,MICRONESIACOUNTRY_ORIGIN,SAINTMARTINFRENCHPART_ORIGIN,SINTMAARTENDUTCHPART_ORIGIN
0,2015,AFGHANISTAN,339432.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
1,2015,ALBANIA,NaN,52031.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
2,2015,ALGERIA,NaN,NaN,239473.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
3,2015,AMERICAN SAMOA,NaN,NaN,NaN,23513.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-23,NaN,NaN,NaN,0,NaN,NaN
4,2015,ANDORRA,NaN,NaN,NaN,NaN,42264.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN


In [5]:
emigrants = emigrants[emigrants.columns.drop(list(emigrants.filter(regex='IMMIGRANTS')))]
emigrants.head()

,YEAR,COUNTRY,EMIGRANTS_FROM_AFGHANISTAN,EMIGRANTS_FROM_ALBANIA,EMIGRANTS_FROM_ALGERIA,EMIGRANTS_FROM_AMERICAN_SAMOA,EMIGRANTS_FROM_ANDORRA,EMIGRANTS_FROM_ANGOLA,EMIGRANTS_FROM_ANGUILLA,EMIGRANTS_FROM_ANTIGUA_AND_BARBUDA,...,EMIGRANTS_FROM_ZAMBIA,EMIGRANTS_FROM_ZIMBABWE,COTEDIVOIRE_DESTINATION,MICRONESIACOUNTRY_DESTINATION,SAINTMARTINFRENCHPART_DESTINATION,SINTMAARTENDUTCHPART_DESTINATION,COTEDIVOIRE_ORIGIN,MICRONESIACOUNTRY_ORIGIN,SAINTMARTINFRENCHPART_ORIGIN,SINTMAARTENDUTCHPART_ORIGIN
0,2015,AFGHANISTAN,-5400916.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
1,2015,ALBANIA,NaN,-1142844.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
2,2015,ALGERIA,NaN,NaN,-1836502.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
3,2015,AMERICAN SAMOA,NaN,NaN,NaN,-1961.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-23,NaN,NaN,NaN,0,NaN,NaN
4,2015,ANDORRA,NaN,NaN,NaN,NaN,-8740.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN
